In [1]:
import requests
import json
import os
import climate
from pprint import pprint
import pandas as pd
import numpy as np
import io
import census
from pprint import pprint

In [2]:
ca = census.CensusData()
pgpw = os.getenv("POSTGRES_PASSWORD")
server, engine = ca.connect_to_postgres(password=pgpw) #need to add create=True if


In [3]:
cities = ca.get_all_cities()

In [6]:
ca.upload_cities_to_postgres(cities, engine=engine)

Uploading region information to database
Finished uploading to database


In [ ]:
pgpw = os.getenv("POSTGRES_PASSWORD")
server, engine = ca.connect_to_postgres(password=pgpw) #need to add create=True if


In [6]:
tmp_df = ca.get_data_for_city("norfolk", years=(2004, 2024))

<!doctype html><html lang="en"><head><title>HTTP Status 404 ? Not Found</title><style type="text/css">body {font-family:Tahoma,Arial,sans-serif;} h1, h2, h3, b {color:white;background-color:#525D76;} h1 {font-size:22px;} h2 {font-size:16px;} h3 {font-size:14px;} p {font-size:12px;} a {color:black;} .line {height:1px;background-color:#525D76;border:none;}</style></head><body><h1>HTTP Status 404 ? Not Found</h1></body></html>
error: error: unknown variable 'DP03_0119PE'
error: error: unknown variable 'DP03_0119PE'
error: error: unknown variable 'DP03_0119PE'
error: error: unknown variable 'DP03_0119PE'
<!doctype html><html lang="en"><head><title>HTTP Status 404 ? Not Found</title><style type="text/css">body {font-family:Tahoma,Arial,sans-serif;} h1, h2, h3, b {color:white;background-color:#525D76;} h1 {font-size:22px;} h2 {font-size:16px;} h3 {font-size:14px;} p {font-size:12px;} a {color:black;} .line {height:1px;background-color:#525D76;border:none;}</style></head><body><h1>HTTP Status

In [7]:
tmp_df

,city,year,male_worker_earnings,female_worker_earnings,pct_below_pov,num_housing_units,num_married_housing_units,avg_household_size,lt_10k_income,gt_10k_lt_15k_income,...,pct_in_retail,pct_in_transportation,pct_in_information,pct_in_finance,pct_in_science,pct_in_education,pct_in_arts,pct_in_pub_ad,state,county
0,norfolk,2009,25986,36696,(X),96216,49423,3.37,85060,7964,...,2.4,12.0,5.2,2.6,5.0,9.3,22.0,5.0,51,710
1,norfolk,2010,34313,32066,11.8,95123,45363,3.69,6526,5104,...,11.6,4.7,2.4,4.7,10.3,22.2,10.5,10.5,51,710
2,norfolk,2011,36202,27373,14.6,95021,49334,3.26,8168,5180,...,13.0,5.2,1.2,4.4,11.8,22.4,10.8,10.8,51,710
3,norfolk,2012,36235,28080,16.4,95435,51735,3.20,9531,5082,...,13.5,4.2,1.5,5.4,11.2,22.5,10.3,9.8,51,710
4,norfolk,2013,35740,30713,17.9,95683,48896,3.32,8479,6299,...,12.0,5.0,1.9,5.2,10.2,25.4,11.4,10.3,51,710
5,norfolk,2014,36485,31778,16.3,96690,50649,3.23,10074,5732,...,12.7,4.9,1.7,5.5,10.2,23.1,12.5,8.1,51,710
6,norfolk,2015,36312,32041,16.1,97141,49634,3.25,9878,4863,...,11.0,5.8,2.0,6.6,10.2,20.4,14.2,10.1,51,710
7,norfolk,2016,37553,31885,16.4,97865,49241,3.26,8361,5402,...,10.6,5.5,1.6,5.2,10.1,24.0,12.6,8.9,51,710
8,norfolk,2017,40724,32066,15.0,97931,50390,3.16,7923,5669,...,11.5,4.9,1.7,6.2,13.7,24.1,12.3,8.2,51,710
9,norfolk,2018,40485,32244,13.7,98218,51345,3.14,8142,4583,...,12.6,4.2,1.8,4.0,10.5,23.9,12.9,9.9,51,710


In [8]:
ca.upload_city_data_to_postgres(tmp_df, engine=engine)
# CITY YEAR will be the primary key(s)
# all other data will follow after that

Uploading city data to the database
Finished uploading city data to database


In [ ]:
# to delete a table,,, weird syntax
from sqlalchemy import MetaData, Table
metadata = MetaData()

# Reflect the existing table
table_name = 'city_data'
table = Table(table_name, metadata, autoload_with=engine)

# Drop the table
table.drop(engine)

In [ ]:
query = """
SELECT * 
FROM cities
"""

counties = list(pd.read_sql_query(query, con=engine)['county'])

for county in counties:
    print("Working on: ", county)
    city_data_df = ca.get_data_for_city(city=None, years=(2009,2024), county=county)
    ca.upload_city_data_to_postgres(city_data_df, engine=engine)



Working on:  003
<!doctype html><html lang="en"><head><title>HTTP Status 404 ? Not Found</title><style type="text/css">body {font-family:Tahoma,Arial,sans-serif;} h1, h2, h3, b {color:white;background-color:#525D76;} h1 {font-size:22px;} h2 {font-size:16px;} h3 {font-size:14px;} p {font-size:12px;} a {color:black;} .line {height:1px;background-color:#525D76;border:none;}</style></head><body><h1>HTTP Status 404 ? Not Found</h1></body></html>
Uploading city data to the database
city_data table does not exist yet, creating now with first command
Finished uploading city data to database
Working on:  013
<!doctype html><html lang="en"><head><title>HTTP Status 404 ? Not Found</title><style type="text/css">body {font-family:Tahoma,Arial,sans-serif;} h1, h2, h3, b {color:white;background-color:#525D76;} h1 {font-size:22px;} h2 {font-size:16px;} h3 {font-size:14px;} p {font-size:12px;} a {color:black;} .line {height:1px;background-color:#525D76;border:none;}</style></head><body><h1>HTTP Status 

In [ ]:

query = """

"""

df = pd.read_sql_query(query, con=engine)
df

ProgrammingError: (psycopg.errors.SyntaxError) syntax error at or near "RENAME"
LINE 2: RENAME TABLE "city_data" TO "city_data1"
        ^
[SQL: 
RENAME TABLE "city_data" TO "city_data1"
]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [ ]:
# ca = census.CensusData()

# pprint(ca.access_economic_data("Norfolk", [2022]))

In [ ]:
from html.parser import HTMLParser
#census_url = "https://api.census.gov/data"
year = "2023"
dataset = "acs"
url = f"https://api.census.gov/data/{year}/acs/acs1/profile"


"""
    # https://api.census.gov/data/2022/acs/acs1/profile/variables.html 

    # ones worth using:

    # economic
    DP03_0093E = estimate for earnings for first year male workers 
    DP03_0094E = estimate for earnings for first year female workers
    DP03_0119PE = estimated percentage of people below the poverty line
    DP04_0001E = estimated housing units
    DP02_0002E = estimated number of married housing units
    DP02_0016E = estimated average household size

    DP03_0052E: estimated total households less than $10,000 income
    DP03_0053E: estimated total households between $10-15k
    DP03_0054E: estimated total households between $15-25k
    DP03_0055E: 25-35k
    DP03_0056E: 35-45k
    DP03_0057E: 50-75k
    DP03_0058E: 75-100k
    DP03_0059E: 100-150k
    DP03_0060E: 150-200k
    DP03_0061E: > 200k

    # relationships
    DP02_0026E = men over 15 y/o who have not married
    DP02_0028E = men over 15 y/o who married, but separated
    DP02_0030E = men over 15 y/o who are divorced
    
    # education (currently)
    DP02_0054E = estimated population of kids in nursery school/preschool
    DP02_0055E = estimated population of kids in kindergarten
    DP02_0056E = estimated population of kids in grades 1-8
    DP02_0057E = estimated population of kids in grades 9-12
    DP02_0058E = estimated population of kids in college/grad school

    # education attainment
    DP02_0060PE = estimated population of 25 years and older who have less than 9th grade education
    DP02_0061PE = estimated population with some high school education
    DP02_0062PE = estimated population with high school diploma
    DP02_0063PE = estimated population with some college, no diploma
    DP02_0064PE = estimated population with associates degrees
    DP02_0065PE  = estimated population with bachelor's degree
    DP02_0066PE = estimated population with graduate degrees

    # unemployement rate
    DP03_0009PE

    # industry
    DP03_0033PE: agriculture, forestry, fishing, hunting, mining
    DP03_0034PE: construction
    DP03_0035PE: manufacturing
    DP03_0036PE: wholesale trade
    DP03_0037PE: retail trade
    DP03_0038PE: transportation, warehousing, utilities
    DP03_0039PE: information
    DP03_0040PE: finance, insurance, real estate, rental, leasing
    DP03_0041PE: professional, scientific, administrative, waste management
    DP03_0042PE: educational services/health care/ social assistance
    DP03_0043PE: arts, entertainment, recreation, accommodation, food services
    DP03_0045PE: public administration

"""
variables = "NAME,DP02_0060PE,DP02_0061PE,DP02_0062PE,DP02_0063PE,DP02_0064PE,DP02_0065PE,DP02_0066PE"
print(url)
params = {"key": os.getenv("CENSUS_KEY"),
           "for": "county:*", 
           "get": variables,
           "in": "state:51"
          }
r = requests.get(url, params=params)
values = list(json.loads(r.text))

df = pd.DataFrame(values[1:], columns=values[0])
# df = df.rename(columns={
#     "DP02_0060PE":"<9th",
#     "DP02_0061PE": "some high school",
#     "DP02_0062PE": "high school",
#     "DP02_0063PE": "some college",
#     "DP02_0064PE": "associates",
#     "DP02_0065PE": "bachelors",
#     "DP02_0066PE": "graduate"
# })
df

In [ ]:
df['NAME'] = df['NAME'].str.lower()
df['NAME'] = df['NAME'].str.replace(" ", "")

In [ ]:
df[df['NAME'].str.contains("norfolk")]

In [ ]:
co = climate.ClimateAgent()
# let's test for all of the cities that we have data on for the Census
# so there will be around 30 "cities" (counties, really) for Virginia that we will have data on
station_id = co.get_station_id("Norfolk, Virginia")
climate_df = co.get_data_for_station_id(station_id=station_id, start_date="2022-01-01", end_date="2022-12-31")